In [237]:
import pandas as pd
import numpy as np
import datetime
import math

In [2]:
path = "/Users/arthurbi/Desktop/g-research-crypto-forecasting/train.csv"
train = pd.read_csv(path)

In [73]:
train.head()


,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [300]:
class GTJA_191:
    def __init__(self, price, asset_id, start_timestamp, end_timestamp):
        price = price[price['Asset_ID']==asset_id]
        price = price[price['timestamp']>=start_timestamp]
        price = price[price['timestamp']<=end_timestamp]
        price = price.set_index(['timestamp'],drop=False)
        self.price     = price
        self.values    = price.values
        self.timestamp = price['timestamp']
        self.id        = price['Asset_ID']
        self.open      = price['Open']
        self.close     = price['Close']
        self.low       = price['Low']
        self.high      = price['High']
        self.volume    = price['Volume']
        self.vwap      = price['VWAP']
        self.amount    = price['Volume']*price['VWAP']
        self.target    = price['Target']
        
    def ret(self):
        res = self.price
        res.insert(10, 'Amount', self.amount, allow_duplicates=False)
        res.insert(11, 'alpha_002', self.alpha_002(), allow_duplicates=False)
        res.insert(12, 'alpha_011', self.alpha_011(), allow_duplicates=False)
        res.insert(13, 'alpha_013', self.alpha_013(), allow_duplicates=False)
        res.insert(14, 'alpha_014', self.alpha_014(), allow_duplicates=False)
        res.insert(15, 'alpha_015', self.alpha_015(), allow_duplicates=False)
        res.insert(16, 'alpha_018', self.alpha_018(), allow_duplicates=False)
        res.insert(17, 'alpha_020', self.alpha_020(), allow_duplicates=False)
        res.insert(18, 'alpha_024', self.alpha_024(), allow_duplicates=False)
        res.insert(19, 'alpha_029', self.alpha_029(), allow_duplicates=False)
        res.insert(20, 'alpha_031', self.alpha_031(), allow_duplicates=False)
        res.insert(21, 'alpha_034', self.alpha_034(), allow_duplicates=False)
        res.insert(22, 'alpha_065', self.alpha_065(), allow_duplicates=False)
        res.insert(23, 'alpha_066', self.alpha_066(), allow_duplicates=False)
        res.insert(24, 'alpha_070', self.alpha_070(), allow_duplicates=False)
        res.insert(25, 'alpha_129', self.alpha_129(), allow_duplicates=False)
        res.insert(26, 'alpha_132', self.alpha_132(), allow_duplicates=False)
        res.insert(27, 'alpha_178', self.alpha_178(), allow_duplicates=False)
        return res
    
    def alpha_002(self):
        alpha = ((self.close-self.low)-(self.high-self.close))/((self.high-self.low)).diff()
        return alpha
    
    def alpha_011(self):
        alpha = ((((self.close-self.low)-(self.high-self.close))/(self.high-self.low))*self.volume).rolling(6).sum()
        return alpha
    
    def alpha_013(self):
#         (((HIGH * LOW)^0.5) - VWAP)
        alpha = (self.high*self.low)**0.5-self.vwap
        return alpha
    
    def alpha_014(self):
#         CLOSE-DELAY(CLOSE,5)
        alpha = self.close - self.close.shift(5)
        return alpha
    
    def alpha_015(self):
#         OPEN/DELAY(CLOSE,1)-1
        alpha = self.open/self.close.shift() - 1
        return alpha
    
    def alpha_018(self):
#         CLOSE/DELAY(CLOSE,5)
        alpha = self.close/self.close.shift(5)
        return alpha
        
    def alpha_020(self):
#         (CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100
        alpha = (self.close-self.close.shift(6))/self.close.shift(6)*100
        return alpha
    
    def alpha_024(self):
        result = self.close - self.close.shift(5)
        result = result.ewm(alpha = 1.0/5).mean()
        alpha = result
        return alpha
    
    def alpha_029(self):
        delay = self.close.shift(6)
        alpha = (self.close-delay)*self.volume/delay
        return alpha
    
    def alpha_031(self):
        rolling_mean = self.close.rolling(12).mean()
        alpha = (self.close-rolling_mean)*100/rolling_mean
        return alpha
    
    def alpha_034(self):
        alpha = self.close.rolling(12).mean()/self.close
        return alpha
        
    def alpha_065(self):
        part1 = self.close.rolling(6)
        alpha = part1.mean()/self.close.shift()
        return alpha_070
    
    def alpha_066(self):
        part1=self.close.rolling(6)
        alpha=(self.close.shift()-part1.mean())/part1.mean()
        return alpha
    
    def alpha_070(self):
        alpha = self.amount.rolling(6).std()
        return alpha
    
    def alpha_129(self):
        data = self.close.diff()
        data[data >= 0] = 0
        data = abs(data)
        alpha = data.rolling(12).sum()
        return alpha
    
    def alpha_132(self):
        alpha = self.amount.rolling(20).mean()
        return alpha
    
    def alpha_178(self):
        alpha = ((self.close-self.close.shift())/self.close.shift()*self.volume).shift()
        return alpha

In [306]:
start_timestamp = 1609459260
end_timestamp = 1632182400
# stock = GTJA_191(train, 12, start_timestamp, end_timestamp)
# res = stock.ret()
# alpha_070 = stock.alpha_070()
# alpha_070.tail()
train.tail(20)
# res.head(20)
data0 = GTJA_191(train, 0, start_timestamp, end_timestamp)
res0 = data0.ret()
data1 = GTJA_191(train, 1, start_timestamp, end_timestamp)
res1 = data1.ret()
data2 = GTJA_191(train, 2, start_timestamp, end_timestamp)
res2 = data2.ret()
data3 = GTJA_191(train, 3, start_timestamp, end_timestamp)
res3 = data3.ret()
data4 = GTJA_191(train, 4, start_timestamp, end_timestamp)
res4 = data4.ret()
data5 = GTJA_191(train, 5, start_timestamp, end_timestamp)
res5 = data5.ret()
data6 = GTJA_191(train, 6, start_timestamp, end_timestamp)
res6 = data6.ret()
data7 = GTJA_191(train, 7, start_timestamp, end_timestamp)
res7 = data7.ret()
data8 = GTJA_191(train, 8, start_timestamp, end_timestamp)
res8 = data8.ret()
data9 = GTJA_191(train, 9, start_timestamp, end_timestamp)
res9 = data9.ret()
data10 = GTJA_191(train, 10, start_timestamp, end_timestamp)
res10 = data10.ret()
data11 = GTJA_191(train, 11, start_timestamp, end_timestamp)
res11 = data11.ret()
data12 = GTJA_191(train, 12, start_timestamp, end_timestamp)
res12 = data12.ret()
data13 = GTJA_191(train, 13, start_timestamp, end_timestamp)
res13 = data13.ret()

# mat = res.values
# print(mat)
final = pd.concat([res0,res1,res2,res3,res4,res5,res6,res7,res8,res9,res10,res11,res12,res13]).dropna()
final = final.reset_index(drop=True, inplace=False)
print(final)

          timestamp  Asset_ID   Count       Open       High        Low  \
0        1609460400         0   231.0  37.118600  37.130200  37.055300   
1        1609460460         0   149.0  37.042250  37.071500  37.025000   
2        1609460520         0    83.0  37.079100  37.097100  37.076100   
3        1609460580         0   111.0  37.090300  37.111300  37.080000   
4        1609460640         0    70.0  37.098900  37.104400  37.082900   
5        1609460700         0    83.0  37.082250  37.120000  37.071000   
6        1609460760         0   186.0  37.098500  37.122400  37.070000   
7        1609460820         0   301.0  37.098400  37.100700  37.031000   
8        1609460880         0   113.0  37.035200  37.073700  37.019000   
9        1609460940         0    80.0  37.048000  37.069000  37.033000   
10       1609461000         0   114.0  37.065300  37.083000  37.058800   
11       1609461060         0   156.0  37.096050  37.112000  37.016700   
12       1609461120         0   100.0 

In [307]:
write_path = "/Users/arthurbi/Desktop/g-research-crypto-forecasting/newdata.csv"
final.to_csv(write_path)